# Auxiliary Telescope Motor Condition Analysis


#### This notebook is used to extract the motor commanded positions, trajectories, velocities, and motor torques for each axis of the Auxiliary Telescope for a given time interval (note that at this time code for Nasmyth 2 does not exist). By inputting a time interval, this notebook allows the rapid determination of what caused the fault, particularily in the case of motor slippage, or if a drive was commanded in an unstable fashion.

##### It is expected that the user interacts with the Bokeh plots to better pinpoint the issue they're searching for. The plot ranges as created will most likely be too large, or contain too much data to be useful with zero manipulation of the axes/zooms etc.

This notebook extracts data from the DM-EFD using [aioinflux](https://aioinflux.readthedocs.io/en/stable/index.html), a Python client for InfluxDB, and proceed with data analysis using Pandas dataframes. 

This is complementaty to the [Chronograf](https://test-chronograf-efd.lsst.codes) interface which we use for time-series visualization.

In addition to `aioinflux`, you'll need to install `pandas`, `numpy` and `matplotlib` to run this notebook.

In [ ]:
import matplotlib
%matplotlib widget
from matplotlib import pylab as plt
import aioinflux
import getpass
import pandas as pd
import asyncio
import numpy as np

from bokeh.plotting import figure, output_notebook, show
from bokeh.models import LinearAxis, Range1d
output_notebook()

We'll access the DM-EFD instance deployed at the AuxTel lab in Tucson. You need to be on site or connected to the NOAO VPN. 

If you are familiar with the AuxTel lab environment, you might be able to authenticate using the generic `saluser`. Ping me at Slack (`@afausti`) if you have any problem.

In [ ]:
username = "saluser"
password = getpass.getpass(f"Password for {username}:")

The following configures the `aioinflux` Python client to connect to the DM-EFD InfluxDB instance. 

In [ ]:
client = aioinflux.InfluxDBClient(host='influxdb-summit-efd.lsst.codes', 
                                  port='443', 
                                  ssl=True, 
                                  username=username, 
                                  password=password,
                                  db='efd')

We can configure the output to be a Pandas dataframe, which is very convenient for data analysis.  Specify a time range for data in `InfluxQL`.  The default is 20hrs ago, but this may need to be changed depending on how recently data was taken.

In [ ]:
client.output = 'dataframe'

# Time_span is in UTC. The EFD is currently indexed by the EFD ingestion time and not yet the send time. This results in small time interval differences.

time_span = "time >= '2019-12-14T02:17:20+00:00' AND time <= '2019-12-14T02:17:50+00:00'"  # Test dataset
time_span = "time >= '2020-01-16T22:50:00+00:00' AND time <= '2020-01-16T23:26:38+00:00'"  # Test dataset

Query each of the measurements we may want to correlate later in the notebook.  Note that this could be done as a single query, but the result is a dictionary of `DataFrames` which I find less convenient to use than named variables corresponding to one `DataFrame` each.

In [ ]:
# This function takes 100Hz data and processes it into a pandas dataframe.
async def get_data_frame(field_base, topic, els=100):
    fields = ", ".join([f'"{field_base}{i}"' for i in range(100)])
    df = await client.query(f'SELECT "cRIO_timestamp", "private_sndStamp", {fields} FROM "efd"."autogen"."{topic}" WHERE {time_span}')

    times = []
    timestamps = []
    vals = []
    step = 1./els
    for row in df.itertuples():
        for i in range(els):
            times.append(row.cRIO_timestamp + i*step)
            timestamps.append((pd.Timestamp(row.cRIO_timestamp, unit='s', tz='GMT') + pd.Timedelta(i*step, unit='s')))
            vals.append(getattr(row, f'{field_base}{i}'))

    return pd.DataFrame({'times':times, field_base:vals}, index=timestamps)

# Azimuth Analysis

In [ ]:
# query angle of azimuth (calculated from 3 individual encoders)
az = await get_data_frame('azimuthCalculatedAngle', 'lsst.sal.ATMCS.mount_AzEl_Encoders')
az_raw = await client.query(f'SELECT "azimuthCalculatedAngle99", "private_sndStamp", "private_rcvStamp" FROM "efd"."autogen"."lsst.sal.ATMCS.mount_AzEl_Encoders" WHERE {time_span}')

In [ ]:
# query the angle commanded by the ATMCS trajectory generator
commanded_az = await get_data_frame('azimuth', 'lsst.sal.ATMCS.trajectory')
commanded_az_raw = await client.query(f'SELECT "azimuth99", "private_sndStamp", "private_rcvStamp" FROM "efd"."autogen"."lsst.sal.ATMCS.trajectory" WHERE {time_span}')

In [ ]:
# query the angle commanded by the AT Pointing Component 
commanded_az_ATPng = await client.query(f'SELECT "azimuth", "private_sndStamp", "private_rcvStamp" FROM "efd"."autogen"."lsst.sal.ATMCS.command_trackTarget" WHERE {time_span}')

In [ ]:
# Useful code snippet on how to convert the time strings to a readable format
pd.to_datetime(commanded_az_ATPng['private_sndStamp'][0], unit='s')

### Plot commanded position (by pointing component), target position by ATMCS, Calculated position by ATMCS

In [ ]:
#derive principal (left) plot axis range
yr_cen=np.median(commanded_az['azimuth'])
dy=1.1*(np.max(commanded_az['azimuth'])- np.min(commanded_az['azimuth']))

p = figure(x_axis_type='datetime', y_range=(yr_cen-dy, yr_cen+dy), plot_width=800, plot_height=400)
p.yaxis.axis_label = "Azimuth (degrees)"
p.xaxis.axis_label = "Time"
p.line(x=commanded_az.index.values, y=commanded_az['azimuth'], color='black', line_width=2, legend_label='ATMCS Commanded Az Trajectory')
p.cross(x=az.index.values, y=az['azimuthCalculatedAngle'], color='red', line_width=2, line_dash='dashed', legend_label='ATMCS Measured Az Position')
p.cross(x=pd.to_datetime(commanded_az_ATPng['private_sndStamp'], unit='s'), y=commanded_az_ATPng['azimuth'], color='green', line_width=2, legend_label='ATPng Target Az')

p.legend.location = 'bottom_left'
p.legend.click_policy = 'hide'
show(p)

## Azimuth motor velocity analysis
#### Note that while tracking, the velocity demand has an offset with respect to the actual velocity, this is normal as the velocity loop has no integrator (just a fixed gain). 

In [ ]:
#Measured Velocity
measured_vel_az1 = await get_data_frame('azimuthMotor1Velocity', 'lsst.sal.ATMCS.measuredMotorVelocity')
measured_vel_az2 = await get_data_frame('azimuthMotor2Velocity', 'lsst.sal.ATMCS.measuredMotorVelocity')
# Commanded Velocity Trajectory
commanded_vel_az = await get_data_frame('azimuthVelocity', 'lsst.sal.ATMCS.trajectory')
# Commanded from Pointing Component
commanded_vel_az_ATPng = await client.query(f'SELECT "azimuthVelocity", "private_sndStamp", "private_rcvStamp" FROM "efd"."autogen"."lsst.sal.ATMCS.command_trackTarget" WHERE {time_span}')

In [ ]:
p = figure(x_axis_type='datetime', y_range=(yr_cen-dy, yr_cen+dy), plot_width=800, plot_height=400)
p.yaxis.axis_label = "Azimuth (degrees)"
p.xaxis.axis_label = "Time"
# Positions
p.line(x=commanded_az.index.values, y=commanded_az['azimuth'], color='black', line_width=2, legend_label='ATMCS Commanded Az Trajectory')
p.cross(x=az.index.values, y=az['azimuthCalculatedAngle'], color='red', line_width=2, line_dash='dashed', legend_label='ATMCS Measured Az Position')
p.cross(x=pd.to_datetime(commanded_az_ATPng['private_sndStamp'], unit='s'), y=commanded_az_ATPng['azimuth'], color='green', line_width=2, legend_label='ATPng Target Az')
# Velocities
p.extra_y_ranges = {'Velocity': Range1d(start=-0.1, end=0.1)}
p.add_layout(LinearAxis(y_range_name='Velocity', axis_label='Velocity'), 'right')
p.line(x=commanded_vel_az.index.values, y=commanded_vel_az['azimuthVelocity'], color='black', alpha=0.7, y_range_name='Velocity', legend_label='ATMCS Commanded Az Velocity Trajectory')
p.line(x=measured_vel_az1.index.values, y=measured_vel_az1['azimuthMotor1Velocity'], color='red', alpha=0.5, y_range_name='Velocity', legend_label='ATMCS Measured Az Velocity Motor 1')
p.line(x=measured_vel_az2.index.values, y=measured_vel_az2['azimuthMotor2Velocity'], color='blue', alpha=0.5, y_range_name='Velocity', legend_label='ATMCS Measured Az Velocity Motor 2')
p.x(x=pd.to_datetime(commanded_vel_az_ATPng['private_sndStamp'], unit='s'), y=commanded_vel_az_ATPng['azimuthVelocity'], y_range_name='Velocity', color='green', line_width=2, legend_label='ATPng Target Az Velocity')

p.legend.location = 'top_left'
p.legend.click_policy = 'hide'
show(p)

### Azimuth Torque Analysis

In [ ]:
# Demanded and Measured Torques
# Motor 1
az_motor1_torque_demand = await get_data_frame('azimuthMotor1Torque', 'lsst.sal.ATMCS.torqueDemand')
az_motor1_torque_measured = await get_data_frame('azimuthMotor1Torque', 'lsst.sal.ATMCS.measuredTorque')
# Motor 2
az_motor2_torque_demand = await get_data_frame('azimuthMotor2Torque', 'lsst.sal.ATMCS.torqueDemand')
az_motor2_torque_measured = await get_data_frame('azimuthMotor2Torque', 'lsst.sal.ATMCS.measuredTorque')

In [ ]:
# Motor 1 Plot
yr_cen=np.median(measured_vel_az1['azimuthMotor1Velocity'])
dy=1.1*(np.max(measured_vel_az1['azimuthMotor1Velocity'])- np.min(measured_vel_az1['azimuthMotor1Velocity']))

p = figure(x_axis_type='datetime', y_range=(yr_cen-dy, yr_cen+dy), plot_width=800, plot_height=400)
p.yaxis.axis_label = "Azimuth Velocity (deg/s)"
p.xaxis.axis_label = "Time"
# Velocities
p.line(x=commanded_vel_az.index.values, y=commanded_vel_az['azimuthVelocity'], color='black', alpha=0.7, legend_label='ATMCS Commanded Az Velocity Trajectory')
p.line(x=measured_vel_az1.index.values, y=measured_vel_az1['azimuthMotor1Velocity'], color='red', alpha=0.5, legend_label='ATMCS Measured Az Velocity Motor 1')
p.x(x=pd.to_datetime(commanded_vel_az_ATPng['private_sndStamp'], unit='s'), y=commanded_vel_az_ATPng['azimuthVelocity'], color='green', line_width=2, legend_label='ATPng Target Az Velocity')
# Torques
yr_cen=np.median(az_motor1_torque_demand['azimuthMotor1Torque'])
dy=(np.max(az_motor1_torque_demand['azimuthMotor1Torque']) - np.min(az_motor1_torque_demand['azimuthMotor1Torque']))/2
p.extra_y_ranges = {'Torque': Range1d(start=yr_cen-dy, end=yr_cen+dy)}
p.add_layout(LinearAxis(y_range_name='Torque', axis_label='Torque [A]'), 'right')
# motor 1
p.line(x=az_motor1_torque_demand.index.values, y=az_motor1_torque_demand['azimuthMotor1Torque'], color='black', alpha=0.7, y_range_name='Torque', legend_label='ATMCS Commanded Motor 1 Torque')
p.line(x=az_motor1_torque_measured.index.values, y=az_motor1_torque_measured['azimuthMotor1Torque'], color='red', alpha=0.5, y_range_name='Torque', legend_label='ATMCS Measured Motor 1 Torque')
# Motor 2


p.legend.location = 'top_left'
p.legend.click_policy = 'hide'
show(p)

In [ ]:
# Motor 2 Plot
yr_cen=np.median(measured_vel_az2['azimuthMotor2Velocity'])
dy=1.1*(np.max(measured_vel_az2['azimuthMotor2Velocity'])- np.min(measured_vel_az2['azimuthMotor2Velocity']))

p = figure(x_axis_type='datetime', y_range=(yr_cen-dy, yr_cen+dy), plot_width=800, plot_height=400)
p.yaxis.axis_label = "Azimuth Velocity (deg/s)"
p.xaxis.axis_label = "Time"
# Velocities
p.line(x=commanded_vel_az.index.values, y=commanded_vel_az['azimuthVelocity'], color='black', alpha=0.7, legend_label='ATMCS Commanded Az Velocity Trajectory')
p.line(x=measured_vel_az2.index.values, y=measured_vel_az2['azimuthMotor2Velocity'], color='blue', alpha=0.5, legend_label='ATMCS Measured Az Velocity Motor 2')
p.x(x=pd.to_datetime(commanded_vel_az_ATPng['private_sndStamp'], unit='s'), y=commanded_vel_az_ATPng['azimuthVelocity'], color='green', line_width=2, legend_label='ATPng Target Az Velocity')
# Torques
yr_cen=np.median(az_motor2_torque_demand['azimuthMotor2Torque'])
dy=(np.max(az_motor2_torque_demand['azimuthMotor2Torque']) - np.min(az_motor2_torque_demand['azimuthMotor2Torque']))/2
p.extra_y_ranges = {'Torque': Range1d(start=yr_cen-dy, end=yr_cen+dy)}
p.add_layout(LinearAxis(y_range_name='Torque', axis_label='Torque [A]'), 'right')
# motor 1
p.line(x=az_motor2_torque_demand.index.values, y=az_motor2_torque_demand['azimuthMotor2Torque'], color='black', alpha=0.7, y_range_name='Torque', legend_label='ATMCS Commanded Motor 2 Torque')
p.line(x=az_motor2_torque_measured.index.values, y=az_motor2_torque_measured['azimuthMotor2Torque'], color='red', alpha=0.5, y_range_name='Torque', legend_label='ATMCS Measured Motor 2 Torque')
# Motor 2


p.legend.location = 'top_left'
p.legend.click_policy = 'hide'
show(p)

# Elevation Axis Analysis

In [ ]:
# Measured Position
el = await get_data_frame('elevationCalculatedAngle', 'lsst.sal.ATMCS.mount_AzEl_Encoders')

In [ ]:
# Commanded Trajectory
commanded_el = await get_data_frame('elevation', 'lsst.sal.ATMCS.trajectory')

In [ ]:
# Pointing Kernel Command
commanded_el_ATPng = await client.query(f'SELECT "elevation", "private_sndStamp", "private_rcvStamp" FROM "efd"."autogen"."lsst.sal.ATMCS.command_trackTarget" WHERE {time_span}')

In [ ]:
# Plot positions
yr_cen=np.median(commanded_el['elevation'])
dy=1.1*(np.max(commanded_el['elevation'])-np.min(commanded_el['elevation']))/2

p = figure(x_axis_type='datetime', y_range=(yr_cen-dy, yr_cen+dy), plot_width=800, plot_height=400)
p.yaxis.axis_label = "Nasmyth Position (degrees)"
p.xaxis.axis_label = "Time"
p.line(x=commanded_el.index.values, y=commanded_el['elevation'], color='black', line_width=2, legend_label='ATMCS Commanded Elevation Trajectory')
p.cross(x=el.index.values, y=el['elevationCalculatedAngle'], color='red', line_width=2, line_dash='dashed', legend_label='ATMCS Elevation Measured Nas1 Position')
p.cross(x=pd.to_datetime(commanded_el_ATPng['private_sndStamp'], unit='s'), y=commanded_el_ATPng['elevation'], color='green', line_width=2, legend_label='ATPng Target Elevation')

p.legend.location = 'bottom_left'
p.legend.click_policy = 'hide'
show(p)

## Elevation Motor velocity Analysis
#### Note that while tracking, the velocity demand has an offset with respect to the actual velocity, this is normal as the velocity loop has no integrator (just a fixed gain). 

In [ ]:
#Measured
measured_vel_el = await get_data_frame('elevationMotorVelocity', 'lsst.sal.ATMCS.measuredMotorVelocity')

In [ ]:
# Commanded
commanded_vel_el = await get_data_frame('elevationVelocity', 'lsst.sal.ATMCS.trajectory')

In [ ]:
# From Pointing
commanded_vel_el_ATPng = await client.query(f'SELECT "elevationVelocity", "private_sndStamp", "private_rcvStamp" FROM "efd"."autogen"."lsst.sal.ATMCS.command_trackTarget" WHERE {time_span}')

In [ ]:
# Plot Velocities and Positions
yr_cen=np.median(commanded_el['elevation'])
dy=1.1*(np.max(commanded_el['elevation'])-np.min(commanded_el['elevation']))/2
# Positions
p = figure(x_axis_type='datetime', y_range=(yr_cen-dy, yr_cen+dy), plot_width=800, plot_height=400)
p.yaxis.axis_label = "Elevation Position (degrees)"
p.xaxis.axis_label = "Time"
p.line(x=commanded_el.index.values, y=commanded_el['elevation'], color='black', line_width=2, legend_label='ATMCS Commanded Elevation Trajectory')
p.cross(x=el.index.values, y=el['elevationCalculatedAngle'], color='red', line_width=2, line_dash='dashed', legend_label='ATMCS Elevation Measured Elevation Position')
p.cross(x=pd.to_datetime(commanded_el_ATPng['private_sndStamp'], unit='s'), y=commanded_el_ATPng['elevation'], color='green', line_width=2, legend_label='ATPng Target Elevation')
# Velocities
yr_cen=np.median(measured_vel_el['elevationMotorVelocity'])
dy=1.1*(np.max(measured_vel_el['elevationMotorVelocity'])- np.min(measured_vel_el['elevationMotorVelocity']))/2

p.extra_y_ranges = {'Velocity': Range1d(start=yr_cen-dy, end=yr_cen+dy)}
p.add_layout(LinearAxis(y_range_name='Velocity', axis_label='Velocity'), 'right')
p.line(x=commanded_vel_el.index.values, y=commanded_vel_el['elevationVelocity'], color='black', alpha=0.7, y_range_name='Velocity', legend_label='ATMCS Commanded Elevation Velocity Trajectory')
p.line(x=measured_vel_el.index.values, y=measured_vel_el['elevationMotorVelocity'], color='red', alpha=0.5, y_range_name='Velocity', legend_label='ATMCS Measured Elevation Velocity')
p.x(x=pd.to_datetime(commanded_vel_el_ATPng['private_sndStamp'], unit='s'), y=commanded_vel_el_ATPng['elevationVelocity'], y_range_name='Velocity', color='green', line_width=2, legend_label='ATPng Target Elevation Velocity')

p.legend.location = 'top_left'
p.legend.click_policy = 'hide'
show(p)

In [ ]:
# Query Torques
el_torque_demand = await get_data_frame('elevationMotorTorque', 'lsst.sal.ATMCS.torqueDemand')
el_torque_measured = await get_data_frame('elevationMotorTorque', 'lsst.sal.ATMCS.measuredTorque')

In [ ]:
yr_cen=np.median(commanded_vel_el['elevationVelocity'])
dy=1.1*(np.max(commanded_vel_el['elevationVelocity']) - np.min(commanded_vel_el['elevationVelocity']))/2

p = figure(x_axis_type='datetime', y_range=(yr_cen-dy, yr_cen+dy), plot_width=800, plot_height=400)
p.yaxis.axis_label = "Azimuth Velocity (deg/s)"
p.xaxis.axis_label = "Time"
# Velocities
p.line(x=commanded_vel_el.index.values, y=commanded_vel_el['elevationVelocity'], color='black', alpha=0.7, legend_label='ATMCS Commanded Elevation Velocity Trajectory')
p.line(x=measured_vel_el.index.values, y=measured_vel_el['elevationMotorVelocity'], color='red', alpha=0.5, legend_label='ATMCS Measured Elevation Velocity')
p.x(x=pd.to_datetime(commanded_vel_el_ATPng['private_sndStamp'], unit='s'), y=commanded_vel_el_ATPng['elevationVelocity'], color='green', 
    line_width=2, legend_label='ATPng Target Elevation Velocity')

# Torques
yr_cen=np.median(el_torque_demand['elevationMotorTorque'])
dy=1.1*(np.max(el_torque_demand['elevationMotorTorque']) - np.min(el_torque_demand['elevationMotorTorque']))/2
p.extra_y_ranges = {'Torque': Range1d(start=yr_cen-dy, end=yr_cen+dy)}
p.add_layout(LinearAxis(y_range_name='Torque', axis_label='Torque [A]'), 'right')
p.line(x=el_torque_demand.index.values, y=el_torque_demand['elevationMotorTorque'], color='black', alpha=0.7, y_range_name='Torque', legend_label='ATMCS Commanded Torque')
p.line(x=el_torque_measured.index.values, y=el_torque_measured['elevationMotorTorque'], color='red', alpha=0.5, y_range_name='Torque', legend_label='ATMCS Measured Torque')


p.legend.location = 'top_left'
p.legend.click_policy = 'hide'
show(p)

# Nasmyth 1 Analysis
#### Note that if Nasmyth 1 was not in use during the selected timestamp this will fail. Nasmyth 2 plots are available below

In [ ]:
# Measured Position
nas1 = await get_data_frame('nasmyth1CalculatedAngle', 'lsst.sal.ATMCS.mount_Nasmyth_Encoders')

In [ ]:
# Commanded Trajectory
commanded_nas1 = await get_data_frame('nasmyth1RotatorAngle', 'lsst.sal.ATMCS.trajectory')

In [ ]:
# Pointing Kernel Command
commanded_nas1_ATPng = await client.query(f'SELECT "nasmyth1RotatorAngle", "private_sndStamp", "private_rcvStamp" FROM "efd"."autogen"."lsst.sal.ATMCS.command_trackTarget" WHERE {time_span}')

## Plot commanded position (by pointing component), target position by ATMCS, Calculated position by ATMCS

In [ ]:
yr_cen=np.median(commanded_nas1['nasmyth1RotatorAngle'])
dy=1.1*(np.max(commanded_nas1['nasmyth1RotatorAngle']) - np.min(commanded_nas1['nasmyth1RotatorAngle']))/2

p = figure(x_axis_type='datetime', y_range=(yr_cen-dy, yr_cen+dy), plot_width=800, plot_height=400)
p.yaxis.axis_label = "Nasmyth Position (degrees)"
p.xaxis.axis_label = "Time"
p.line(x=commanded_nas1.index.values, y=commanded_nas1['nasmyth1RotatorAngle'], color='black', line_width=2, legend_label='ATMCS Commanded Nas1 Trajectory')
p.cross(x=nas1.index.values, y=nas1['nasmyth1CalculatedAngle'], color='red', line_width=2, line_dash='dashed', legend_label='ATMCS Nas1 Measured Nas1 Position')
#p.cross(x=pd.to_datetime(commanded_nas1_ATPng['private_sndStamp'], unit='s'), y=commanded_nas1_ATPng['nasmyth1RotatorAngle'], color='green', line_width=5, legend_label='ATPng Target Nas1')

p.legend.location = 'bottom_left'
p.legend.click_policy = 'hide'
show(p)

## Look at Nasmyth 1 motor velocities
#### Note that while tracking, the velocity demand has an offset with respect to the actual velocity, this is normal as the velocity loop has no integrator (just a fixed gain). 

In [ ]:
#Measured
measured_vel_nas1 = await get_data_frame('nasmyth1MotorVelocity', 'lsst.sal.ATMCS.measuredMotorVelocity')
# Commanded
commanded_vel_nas1 = await get_data_frame('nasmyth1RotatorAngleVelocity', 'lsst.sal.ATMCS.trajectory')
# From Pointing
commanded_vel_nas1_ATPng = await client.query(f'SELECT "nasmyth1RotatorAngleVelocity", "private_sndStamp", "private_rcvStamp" FROM "efd"."autogen"."lsst.sal.ATMCS.command_trackTarget" WHERE {time_span}')

In [ ]:
yr_cen=np.median(commanded_nas1['nasmyth1RotatorAngle'])
dy=1.1*(np.max(commanded_nas1['nasmyth1RotatorAngle']) - np.min(commanded_nas1['nasmyth1RotatorAngle']))/2

# Positions
p = figure(x_axis_type='datetime', y_range=(yr_cen-dy, yr_cen+dy), plot_width=800, plot_height=400)
p.yaxis.axis_label = "Nasmyth Position (degrees)"
p.xaxis.axis_label = "Time"
p.line(x=commanded_nas1.index.values, y=commanded_nas1['nasmyth1RotatorAngle'], color='black', line_width=2, legend_label='ATMCS Commanded Nas1 Trajectory')
p.cross(x=nas1.index.values, y=nas1['nasmyth1CalculatedAngle'], color='red', line_width=2, line_dash='dashed', legend_label='ATMCS Nas1 Measured Nas1 Position')
#p.cross(x=pd.to_datetime(commanded_nas1_ATPng['private_sndStamp'], unit='s'), y=commanded_nas1_ATPng['nasmyth1RotatorAngle'], color='green', line_width=2, legend_label='ATPng Target Nas1')

# Velocities
p.extra_y_ranges = {'Velocity': Range1d(start=-0.1, end=0.1)}
p.add_layout(LinearAxis(y_range_name='Velocity', axis_label='Velocity'), 'right')
p.line(x=commanded_vel_nas1.index.values, y=commanded_vel_nas1['nasmyth1RotatorAngleVelocity'], color='black', alpha=0.5, y_range_name='Velocity', legend_label='ATMCS Commanded Nas1 Velocity Trajectory')
p.line(x=measured_vel_nas1.index.values, y=measured_vel_nas1['nasmyth1MotorVelocity'], color='blue', alpha=0.5, y_range_name='Velocity', legend_label='ATMCS Measured Nas1 Velocity Motor 1')
#p.x(x=pd.to_datetime(commanded_vel_nas1_ATPng['private_sndStamp'], unit='s'), y=commanded_vel_nas1_ATPng['nasmyth1RotatorAngleVelocity'], y_range_name='Velocity', color='green', line_width=2, legend_label='ATPng Target Nas1 Velocity')

p.legend.location = 'top_left'
p.legend.click_policy = 'hide'
show(p)

## Look at Nasmyth 1 Torques as a function of position and velocity

In [ ]:
# Query Torques
nas1_torque_demand = await get_data_frame('nasmyth1MotorTorque', 'lsst.sal.ATMCS.torqueDemand')
nas1_torque_measured = await get_data_frame('nasmyth1MotorTorque', 'lsst.sal.ATMCS.measuredTorque')

In [ ]:
# Plot Position and torque as a function of time
yr_cen=np.median(commanded_nas1['nasmyth1RotatorAngle'])
dy=1.1*(np.max(commanded_nas1['nasmyth1RotatorAngle']) - np.min(commanded_nas1['nasmyth1RotatorAngle']))/2

# Positions
p = figure(x_axis_type='datetime', y_range=(yr_cen-dy, yr_cen+dy), plot_width=800, plot_height=400)
p.yaxis.axis_label = "Nasmyth Position (degrees)"
p.xaxis.axis_label = "Time"

p.line(x=commanded_nas1.index.values, y=commanded_nas1['nasmyth1RotatorAngle'], color='black', line_width=2, legend_label='ATMCS Commanded Nas1 Trajectory')
p.cross(x=nas1.index.values, y=nas1['nasmyth1CalculatedAngle'], color='red', line_width=2, line_dash='dashed', legend_label='ATMCS Nas1 Measured Nas1 Position')
#p.cross(x=pd.to_datetime(commanded_nas1_ATPng['private_sndStamp'], unit='s'), y=commanded_nas1_ATPng['nasmyth1RotatorAngle'], color='green', line_width=2, legend_label='ATPng Target Nas1')

# Torques
yr_cen=np.median(nas1_torque_measured['nasmyth1MotorTorque'])
dy=1.1*(np.max(nas1_torque_measured['nasmyth1MotorTorque']) - np.min(nas1_torque_measured['nasmyth1MotorTorque']))/2

p.extra_y_ranges = {'Torque': Range1d(yr_cen-dy, yr_cen+dy)}
p.add_layout(LinearAxis(y_range_name='Torque', axis_label='Torque'), 'right')
p.line(x=nas1_torque_demand.index.values, y=nas1_torque_demand['nasmyth1MotorTorque'], color='black', alpha=0.5, y_range_name='Torque', legend_label='ATMCS Demanded Nas1 Torque')
p.line(x=nas1_torque_measured.index.values, y=nas1_torque_measured['nasmyth1MotorTorque'], color='blue', alpha=0.5, y_range_name='Torque', legend_label='ATMCS Measured Nas1 Torque')

p.legend.location = 'top_left'
p.legend.click_policy = 'hide'
show(p)

In [ ]:
# Plot velocity and torque as a function of time
yr_cen=np.median(commanded_vel_nas1['nasmyth1RotatorAngleVelocity'])
dy=1.1*(np.max(commanded_vel_nas1['nasmyth1RotatorAngleVelocity']) - np.min(commanded_vel_nas1['nasmyth1RotatorAngleVelocity']))/2

# Velocities
p = figure(x_axis_type='datetime', y_range=(yr_cen-dy, yr_cen+dy), plot_width=800, plot_height=400)
p.yaxis.axis_label = "Nasmyth Velocity (deg/s)"
p.xaxis.axis_label = "Time"
p.line(x=commanded_vel_nas1.index.values, y=commanded_vel_nas1['nasmyth1RotatorAngleVelocity'], color='black', alpha=0.5, legend_label='ATMCS Commanded Nas1 Velocity Trajectory')
p.line(x=measured_vel_nas1.index.values, y=measured_vel_nas1['nasmyth1MotorVelocity'], color='red', alpha=0.5, legend_label='ATMCS Measured Nas1 Velocity Motor 1')
#p.x(x=pd.to_datetime(commanded_vel_nas1_ATPng['private_sndStamp'], unit='s'), y=commanded_vel_nas1_ATPng['nasmyth1RotatorAngleVelocity'], color='green', line_width=2, legend_label='ATPng Target Nas1 Velocity')


# Torques
yr_cen=np.median(nas1_torque_measured['nasmyth1MotorTorque'])
dy=1.1*(np.max(nas1_torque_measured['nasmyth1MotorTorque']) - np.min(nas1_torque_measured['nasmyth1MotorTorque']))/2

p.extra_y_ranges = {'Torque': Range1d(yr_cen-dy, yr_cen+dy)}
p.add_layout(LinearAxis(y_range_name='Torque', axis_label='Torque'), 'right')
p.line(x=nas1_torque_demand.index.values, y=nas1_torque_demand['nasmyth1MotorTorque'], color='black', alpha=0.5, y_range_name='Torque', legend_label='ATMCS Demanded Nas1 Torque')
p.line(x=nas1_torque_measured.index.values, y=nas1_torque_measured['nasmyth1MotorTorque'], color='blue', alpha=0.5, y_range_name='Torque', legend_label='ATMCS Measured Nas1 Torque')

p.legend.location = 'top_left'
p.legend.click_policy = 'hide'
show(p)

# Nasmyth 2 Analysis
#### Note that if Nasmyth 1 was not in use during the selected timestamp this will fail. Nasmyth 1 plots are available above

In [ ]:
# Measured Position
nas2 = await get_data_frame('nasmyth2CalculatedAngle', 'lsst.sal.ATMCS.mount_Nasmyth_Encoders')

In [ ]:
# Commanded Trajectory
commanded_nas2 = await get_data_frame('nasmyth2RotatorAngle', 'lsst.sal.ATMCS.trajectory')

In [ ]:
# Pointing Kernel Command
commanded_nas2_ATPng = await client.query(f'SELECT "nasmyth2RotatorAngle", "private_sndStamp", "private_rcvStamp" FROM "efd"."autogen"."lsst.sal.ATMCS.command_trackTarget" WHERE {time_span}')

## Plot commanded position (by pointing component), target position by ATMCS, Calculated position by ATMCS

In [ ]:
yr_cen=np.median(commanded_nas2['nasmyth2RotatorAngle'])
dy=1.1*(np.max(commanded_nas2['nasmyth2RotatorAngle']) - np.min(commanded_nas2['nasmyth2RotatorAngle']))/2

p = figure(x_axis_type='datetime', y_range=(yr_cen-dy, yr_cen+dy), plot_width=800, plot_height=400)
p.yaxis.axis_label = "Nasmyth Position (degrees)"
p.xaxis.axis_label = "Time"
p.line(x=commanded_nas2.index.values, y=commanded_nas2['nasmyth2RotatorAngle'], color='black', line_width=2, legend_label='ATMCS Commanded Nas2 Trajectory')
p.cross(x=nas2.index.values, y=nas2['nasmyth2CalculatedAngle'], color='red', line_width=2, line_dash='dashed', legend_label='ATMCS Nas1 Measured Nas2 Position')#
#p.cross(x=pd.to_datetime(commanded_nas2_ATPng['private_sndStamp'], unit='s'), y=commanded_nas2_ATPng['nasmyth2RotatorAngle'], color='green', line_width=5, legend_label='ATPng Target Nas2')

p.legend.location = 'bottom_left'
p.legend.click_policy = 'hide'
show(p)

## Look at Nasmyth 2 motor velocities
#### Note that while tracking, the velocity demand has an offset with respect to the actual velocity, this is normal as the velocity loop has no integrator (just a fixed gain). 

In [ ]:
#Measured
measured_vel_nas2 = await get_data_frame('nasmyth2MotorVelocity', 'lsst.sal.ATMCS.measuredMotorVelocity')
# Commanded
commanded_vel_nas2 = await get_data_frame('nasmyth2RotatorAngleVelocity', 'lsst.sal.ATMCS.trajectory')
# From Pointing
commanded_vel_nas2_ATPng = await client.query(f'SELECT "nasmyth2RotatorAngleVelocity", "private_sndStamp", "private_rcvStamp" FROM "efd"."autogen"."lsst.sal.ATMCS.command_trackTarget" WHERE {time_span}')

In [ ]:
# Positions
p = figure(x_axis_type='datetime', y_range=(yr_cen-dy, yr_cen+dy), plot_width=800, plot_height=400)
p.line(x=commanded_nas2.index.values, y=commanded_nas2['nasmyth2RotatorAngle'], color='black', line_width=2, legend_label='ATMCS Commanded Nas2 Trajectory')
p.cross(x=nas2.index.values, y=nas2['nasmyth2CalculatedAngle'], color='red', line_width=2, line_dash='dashed', legend_label='ATMCS Nas1 Measured Nas2 Position')
#p.cross(x=pd.to_datetime(commanded_nas2_ATPng['private_sndStamp'], unit='s'), y=commanded_nas2_ATPng['nasmyth2RotatorAngle'], color='green', line_width=2, legend_label='ATPng Target Nas2')

# Velocities
p.extra_y_ranges = {'Velocity': Range1d(start=-0.1, end=0.1)}
p.add_layout(LinearAxis(y_range_name='Velocity', axis_label='Velocity'), 'right')
p.line(x=commanded_vel_nas2.index.values, y=commanded_vel_nas2['nasmyth2RotatorAngleVelocity'], color='black', alpha=0.5, y_range_name='Velocity', legend_label='ATMCS Commanded Nas2 Velocity Trajectory')
p.line(x=measured_vel_nas2.index.values, y=measured_vel_nas2['nasmyth2MotorVelocity'], color='blue', alpha=0.5, y_range_name='Velocity', legend_label='ATMCS Measured Nas2 Velocity Motor 1')
#p.x(x=pd.to_datetime(commanded_vel_nas2_ATPng['private_sndStamp'], unit='s'), y=commanded_vel_nas2_ATPng['nasmyth2RotatorAngleVelocity'], y_range_name='Velocity', color='green', line_width=2, legend_label='ATPng Target Nas1 Velocity')

p.legend.location = 'top_left'
p.legend.click_policy = 'hide'
show(p)

In [ ]:
# Query Torques
nas2_torque_demand = await get_data_frame('nasmyth2MotorTorque', 'lsst.sal.ATMCS.torqueDemand')
nas2_torque_measured = await get_data_frame('nasmyth2MotorTorque', 'lsst.sal.ATMCS.measuredTorque')

In [ ]:
# Plot Position and torque as a function of time
yr_cen=np.median(commanded_nas2['nasmyth2RotatorAngle'])
dy=1.1*(np.max(commanded_nas2['nasmyth2RotatorAngle']) - np.min(commanded_nas2['nasmyth2RotatorAngle']))/2

# Positions
p = figure(x_axis_type='datetime', y_range=(yr_cen-dy, yr_cen+dy), plot_width=800, plot_height=400)
p.yaxis.axis_label = "Nasmyth Position (degrees)"
p.xaxis.axis_label = "Time"

p.line(x=commanded_nas2.index.values, y=commanded_nas2['nasmyth2RotatorAngle'], color='black', line_width=2, legend_label='ATMCS Commanded Nas2 Trajectory')
p.cross(x=nas2.index.values, y=nas2['nasmyth2CalculatedAngle'], color='red', line_width=2, line_dash='dashed', legend_label='ATMCS Measured Nas2 Position')
#p.cross(x=pd.to_datetime(commanded_nas2_ATPng['private_sndStamp'], unit='s'), y=commanded_nas2_ATPng['nasmyth2RotatorAngle'], color='green', line_width=2, legend_label='ATPng Target Nas2')

# Torques
yr_cen=np.median(nas2_torque_measured['nasmyth2MotorTorque'])
dy=1.1*(np.max(nas2_torque_measured['nasmyth2MotorTorque']) - np.min(nas2_torque_measured['nasmyth2MotorTorque']))/2

p.extra_y_ranges = {'Torque': Range1d(yr_cen-dy, yr_cen+dy)}
p.add_layout(LinearAxis(y_range_name='Torque', axis_label='Torque'), 'right')
p.line(x=nas2_torque_demand.index.values, y=nas2_torque_demand['nasmyth2MotorTorque'], color='black', alpha=0.5, y_range_name='Torque', legend_label='ATMCS Demanded Nas2 Torque')
p.line(x=nas2_torque_measured.index.values, y=nas2_torque_measured['nasmyth2MotorTorque'], color='blue', alpha=0.5, y_range_name='Torque', legend_label='ATMCS Measured Nas2 Torque')

p.legend.location = 'top_left'
p.legend.click_policy = 'hide'
show(p)

In [ ]:
# Plot velocity and torque as a function of time
yr_cen=np.median(commanded_vel_nas2['nasmyth2RotatorAngleVelocity'])
dy=1.1*(np.max(commanded_vel_nas2['nasmyth2RotatorAngleVelocity']) - np.min(commanded_vel_nas2['nasmyth2RotatorAngleVelocity']))/2

# Velocities
p = figure(x_axis_type='datetime', y_range=(yr_cen-dy, yr_cen+dy), plot_width=800, plot_height=400)
p.yaxis.axis_label = "Nasmyth Velocity (deg/s)"
p.xaxis.axis_label = "Time"
p.line(x=commanded_vel_nas2.index.values, y=commanded_vel_nas2['nasmyth2RotatorAngleVelocity'], color='black', alpha=0.5, legend_label='ATMCS Commanded Nas2 Velocity Trajectory')
p.line(x=measured_vel_nas2.index.values, y=measured_vel_nas2['nasmyth2MotorVelocity'], color='red', alpha=0.5, legend_label='ATMCS Measured Nas2 Velocity Motor 1')
#p.x(x=pd.to_datetime(commanded_vel_nas2_ATPng['private_sndStamp'], unit='s'), y=commanded_vel_nas2_ATPng['nasmyth2RotatorAngleVelocity'], color='green', line_width=2, legend_label='ATPng Target Nas2 Velocity')


# Torques
yr_cen=np.median(nas2_torque_measured['nasmyth2MotorTorque'])
dy=1.1*(np.max(nas2_torque_measured['nasmyth2MotorTorque']) - np.min(nas2_torque_measured['nasmyth2MotorTorque']))/2

p.extra_y_ranges = {'Torque': Range1d(yr_cen-dy, yr_cen+dy)}
p.add_layout(LinearAxis(y_range_name='Torque', axis_label='Torque'), 'right')
p.line(x=nas2_torque_demand.index.values, y=nas2_torque_demand['nasmyth2MotorTorque'], color='black', alpha=0.5, y_range_name='Torque', legend_label='ATMCS Demanded Nas2 Torque')
p.line(x=nas2_torque_measured.index.values, y=nas2_torque_measured['nasmyth2MotorTorque'], color='blue', alpha=0.5, y_range_name='Torque', legend_label='ATMCS Measured Nas2 Torque')

p.legend.location = 'top_left'
p.legend.click_policy = 'hide'
show(p)